# Preprocessing MAVEN with NIAK dev-0.14.0

## Maven DICOM to MINC conversion 

### Converting to minc in batch 

First go to https://portal.cbrain.mcgill.ca/ and dowload the raw dicom

#######24 june 2016 ##########

this is the tree of maven data after download from CBRAIN 
.
├── A1470_316392_MRI10years.tar.gz
├── A1522_244094_MRI10years.tar.gz
├── A1568_797066_MRI10years.tar.gz
├── A1587_600537_MRI10years.tar.gz
├── A1611_205804_MRI10years.tar.gz
├── A1623_856310_MRI10years.tar.gz
├── A1642_299799_MRI10years.tar.gz
├── A1731_213931_MRI10years.tar.gz
├── A1773_311353_MRI10years.tar.gz
├── A1790_239854_MRI10years.tar.gz
├── A1820_767332_MRI10years.tar.gz
├── A1820-B_435858_MRI10years.tar.gz
├── B1658_363107_MRI10years.tar.gz
├── B1687_220180_MRI10years.tar.gz
├── B1692_518881_MRI10years.tar.gz
├── B1783_464014_MRI10years.tar.gz
├── C2063_304633_MRI10years.tar.gz
├── C2065_314679_MRI10years.tar.gz
├── C2106_899371_MRI10years.tar.gz
├── D2026_444020_MRI10years.tar.gz
├── D2098_720379_MRI10years.tar.gz
├── D2228_535094_MRI10years.tar.gz
├── D2354_382216_MRI10years.tar.gz
├── D2358_485813_MRI10years.tar.gz
├── D2410_292645_MRI10years.tar.gz
├── D2422_549151_MRI10years.tar.gz
├── D2443_690805_MRI10years.tar.gz
├── D2443-B_373052_MRI10years.tar.gz
├── D2545_539066_MRI10years.tar.gz
├── D2568_538326_MRI10years.tar.gz
├── D2570_262417_MRI10years.tar.gz
├── D2583_673074_MRI10years.tar.gz
└── P0003_723687_MRI10years.tar.gz

0 directories, 33 files


this is the tree of maven data after unzip 
.
├── 2015_01_11_KARSHE_A1790_20150111_154555981
├── 2015_01_17_karshe_D2570_20150117_110918112
├── 2015_02_07_karshe_C2106_20150207_124345655
├── 2015_03_21_karshe_B1692_20150321_111411350
├── A1470_316392_MRI10years
├── A1522_244094_MRI10years
├── A1568_797066_MRI10years
├── A1587_600537_MRI10years
├── A1611_205804_MRI10years
├── A1623_856310_MRI10years
├── A1642_299799_MRI10years
├── A1731_213931_MRI10years
├── A1773_311353_MRI10years
├── A1820_767332_MRI10years
├── A1820-B_435858_MRI10years
├── B1658_363107_MRI10years
├── B1687_220180_MRI10years
├── B1783_464014_MRI10years
├── C2063_304633_MRI10years
├── C2065_314679_MRI10years
├── D2026_444020_MRI10years
├── D2098_720379_MRI10years
├── D2228_535094_MRI10years
├── D2354_382216_MRI10years
├── D2358_485813_MRI10years
├── D2410_292645_MRI10years
├── D2422_549151_MRI10years
├── D2443_690805_MRI10years
├── D2443-B_373052_MRI10years
├── D2545_539066_MRI10years
├── D2568_538326_MRI10years
├── D2583_673074_MRI10years
└── P0003_723687_MRI10years

33 directories, 0 file

###############################


In this step I simply convert all dicom to a temporary folder

In [ ]:
%% script to convert maven database from dicom to mnc

clear
path_dcm = '/media/yassinebha/database24/MAVEN_06_2016/raw_dcm/';
path_mnc = '/media/yassinebha/database24/MAVEN_06_2016/raw_mnc/';
opt_pipe.path_logs = [path_mnc 'logs_conversion'];
list_subject = dir(path_dcm);
list_subject = {list_subject(3:end).name};

nb_subject = length(list_subject);
pipeline = struct();
for num_s = 1:nb_subject
    subject_folder = list_subject{num_s};
    subject_ID = subject_folder(1:(strfind(subject_folder,'_')(1))-1);
    subject_ID_tmp = subject_ID;
    subject_ID(strfind(subject_ID,'-')) = [];
    fprintf('Subject %s\n',subject_ID);
    path_read      = [path_dcm subject_folder filesep];        
    path_write     = [path_mnc subject_ID filesep];
    path_tmp       = [path_write 'tmp' filesep];
    
    % create folders
    niak_mkdir([path_write 'tmp/']);
    niak_mkdir([path_write 'func/']);
    niak_mkdir([path_write 'anat/']);
    
    % convert all to mnc in the tmp folder
    instr_dcm2mnc = ['dcm2mnc -dname '''' ' path_read '* ' path_tmp];
   
    % start jobs
    name_job = sprintf('%s',subject_ID);
    pipeline.(name_job).opt.instr_conv = instr_dcm2mnc;
    pipeline.(name_job).command = 'system(opt.instr_conv_final)';
end
opt_pipe.max_queued = 8;
psom_run_pipeline(pipeline,opt_pipe)

### Running preprocessing script

In [1]:
%% script to check availabilty of images in maven database and arrange them 
%% according to the bids format

clear
path_mnc = '/media/yassinebha/database24/MAVEN_06_2016/raw_mnc/';
list_subject = dir(path_mnc);
list_subject = {list_subject(3:end).name};
list_subject = list_subject(~ismember(list_subject,{'.','..','octave-wokspace','octave-core','qc_report.csv','logs_conversion'}));
for ll = 1:length(list_subject)
    subject_ID = list_subject{ll};
    fprintf('\n  Subject %s\n',subject_ID);
    path_in = [path_mnc subject_ID '/tmp/'];
    list_file = dir(path_in);
    list_file = {list_file(3:end).name};
    list_file = list_file(~ismember(list_file,{'.','..','octave-wokspace','octave-core'}));
    flag_exist = zeros(1,4);
    for ii = 1:length(list_file)
        [status,output] = system(['mincheader ' path_in list_file{ii} ' |grep acquisition:protocol']);
        output  = strtrim(output);
        pos = strfind (output,'"');
        pattern = output(pos(1):pos(2));
        switch pattern
               case '"RSN_1(ep2d_64)"'
               system(['scp -r ' path_in list_file{ii} ' ' path_mnc subject_ID '/func/' subject_ID '_task-rest1_run-01_bold.mnc' ]);
               fprintf('Rest1 file "%s" found for subject %s\n',list_file{ii},subject_ID);
               flag_exist(1,1) = true;
               
               case '"RSN_2(ep2d_64)"'
               system(['scp -r ' path_in list_file{ii}  ' ' path_mnc subject_ID '/func/' subject_ID '_task-rest2inscape_run-02_bold.mnc' ]);
               fprintf('Rest2 file "%s" found for subject %s\n',list_file{ii},subject_ID);
               flag_exist(1,2) = true;
               
               case '"RSN_3(ep2d_64)"'
               system(['scp -r ' path_in list_file{ii}  ' ' path_mnc subject_ID '/func/' subject_ID '_task-rest3_run-03_bold.mnc' ]);
               fprintf('Rest3 file "%s" found for subject %s\n',list_file{ii},subject_ID);
               flag_exist(1,3) = true;
               
               case '"MPRAGE (t1_mprage)"'
               system(['scp -r ' path_in list_file{ii}  ' ' path_mnc subject_ID '/anat/' subject_ID '_T1w.mnc' ]);
               fprintf('MPRAGE file "%s" found for subject %s\n',list_file{ii},subject_ID);
               flag_exist(1,4) = true;
       end
    end
    if sum(flag_exist(1,1:3)) == 0
       fprintf('WARNING:subject %s has some missing run\n', subject_ID);
    end
    if flag_exist(1,4) == 0
       fprintf('FATAL-WARNING:subject %s missing anat file\n', subject_ID);
    end
end


  Subject A1470
MPRAGE file "a1470_316392_mri10years_20140331_172609_11_mri.mnc" found for subject A1470
MPRAGE file "a1470_316392_mri10years_20140331_172609_13_mri.mnc" found for subject A1470
Rest1 file "a1470_316392_mri10years_20140331_172609_4_mri.mnc" found for subject A1470
Rest2 file "a1470_316392_mri10years_20140331_172609_5_mri.mnc" found for subject A1470
Rest3 file "a1470_316392_mri10years_20140331_172609_6_mri.mnc" found for subject A1470
MPRAGE file "a1470_316392_mri10years_20140331_172609_9_mri.mnc" found for subject A1470

  Subject A1522
MPRAGE file "a1522_244094_mri10years_20140831_131945_10_mri.mnc" found for subject A1522
Rest1 file "a1522_244094_mri10years_20140831_131945_4_mri.mnc" found for subject A1522
Rest2 file "a1522_244094_mri10years_20140831_131945_5_mri.mnc" found for subject A1522
Rest3 file "a1522_244094_mri10years_20140831_131945_6_mri.mnc" found for subject A1522
MPRAGE file "a1522_244094_mri10years_20140831_131945_9_mri.mnc" found for subject A1522



Subject 

In [6]:
clear all
addpath(genpath('/home/yassinebha/niak-dev-0.14.0'))
%%%%%%%%%%%%%%%%%%%%%
%% Parameters
%%%%%%%%%%%%%%%%%%%%%
task  = 'inscape_rest';
%% Setting input/output files 
[status,cmdout] = system ('uname -n');
server          = strtrim(cmdout);
if strfind(server,'lg-1r') % This is guillimin
    root_path = '/gs/scratch/yassinebha/MAVEN/';
    fprintf ('server: %s (Guillimin) \n ',server)
    my_user_name = getenv('USER');
elseif strfind(server,'ip05') % this is mammouth
    root_path = '/mnt/parallel_scratch_ms2_wipe_on_april_2015/pbellec/benhajal/MAVEN/';
    fprintf ('server: %s (Mammouth) \n',server)
    my_user_name = getenv('USER');
else
    switch server
        case 'peuplier' % this is peuplier
        root_path = '/media/database6/MAVEN/';
        fprintf ('server: %s\n',server)
        my_user_name = getenv('USER');
        
        case 'noisetier' % this is noisetier
        root_path = '/media/yassinebha/database24/MAVEN_06_2016/';
        fprintf ('server: %s\n',server)
        my_user_name = getenv('USER');
    end
end
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% grab raw_data
path_raw = [root_path 'raw_mnc/'];
list_subject = dir(path_raw);
list_subject = {list_subject.name};
list_subject = list_subject(~ismember(list_subject,{'.','..','octave-wokspace','octave-core','qc_report.csv','D2026','logs_conversion'}));
for num_s = 1:length(list_subject)
    subject_ID = list_subject{num_s};
    fprintf('Subject %s\n',subject_ID);
    path_anat = [path_raw subject_ID filesep 'anat/'];
    path_func = [path_raw subject_ID filesep 'func/'];
    files_in.(subject_ID).anat = [ path_anat subject_ID '_T1w.mnc'];
    
    files_in.(subject_ID).fmri.session1.rest1 = [path_func subject_ID '_task-rest1_run-01_bold.mnc'];
              
    files_in.(subject_ID).fmri.session1.inscape = [path_func subject_ID '_task-rest2inscape_run-02_bold.mnc'];
              
    files_in.(subject_ID).fmri.session1.rest2 = [path_func subject_ID '_task-rest3_run-03_bold.mnc'];
              
end


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%% Setting input/output files %%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%% General
opt.folder_out  = [ root_path 'fmri_preprocess_' upper(task) ];    % Where to store the results
opt.size_output = 'quality_control';                             % The amount of outputs that are generated by the pipeline. 'all' will keep intermediate outputs, 'quality_control' will only keep the quality control outputs.
%% Slice timing correction (niak_brick_slice_timing)
opt.slice_timing.type_acquisition = 'interleaved descending'; % Slice timing order (available options : 'sequential ascending', 'sequential descending', 'interleaved ascending', 'interleaved descending')
opt.slice_timing.type_scanner     = 'Siemens';                % Scanner manufacturer. Only the value 'Siemens' will actually have an impact
opt.slice_timing.delay_in_tr      = 0.05;                       % The delay in TR ("blank" time between two volumes)
opt.slice_timing.suppress_vol     = 0;                       % Number of dummy scans to suppress.
opt.slice_timing.flag_nu_correct  = 1;                       % Apply a correction for non-uniformities on the EPI volumes (1: on, 0: of). This is particularly important for 32-channels coil.
opt.slice_timing.arg_nu_correct   = '-distance 200';         % The distance between control points for non-uniformity correction (in mm, lower values can capture faster varying slow spatial drifts).
opt.slice_timing.flag_center      = 0;                       % Set the origin of the volume at the center of mass of a brain mask. This is useful only if the voxel-to-world transformation from the DICOM header has somehow been damaged. This needs to be assessed on the raw images.
opt.slice_timing.flag_skip        = 0;                       % Skip the slice timing (0: don't skip, 1 : skip). Note that only the slice timing corretion portion is skipped, not all other effects such as FLAG_CENTER or FLAG_NU_CORRECT
 
% Motion estimation (niak_pipeline_motion)
opt.motion.session_ref  = 'session1'; % The session that is used as a reference. In general, use the session including the acqusition of the T1 scan.

% resampling in stereotaxic space
opt.resample_vol.interpolation = 'trilinear'; % The resampling scheme. The fastest and most robust method is trilinear. 
opt.resample_vol.voxel_size    = [3 3 3];     % The voxel size to use in the stereotaxic space
opt.resample_vol.flag_skip     = 0;           % Skip resampling (data will stay in native functional space after slice timing/motion correction) (0: don't skip, 1 : skip)

% Linear and non-linear fit of the anatomical image in the stereotaxic
% space (niak_brick_t1_preprocess)
opt.t1_preprocess.nu_correct.arg = '-distance 75'; % Parameter for non-uniformity correction. 200 is a suggested value for 1.5T images, 75 for 3T images. If you find that this stage did not work well, this parameter is usually critical to improve the results.

% Temporal filtering (niak_brick_time_filter)
opt.time_filter.hp = 0.01; % Cut-off frequency for high-pass filtering, or removal of low frequencies (in Hz). A cut-off of -Inf will result in no high-pass filtering.
opt.time_filter.lp = Inf;  % Cut-off frequency for low-pass filtering, or removal of high frequencies (in Hz). A cut-off of Inf will result in no low-pass filtering.

% Regression of confounds and scrubbing (niak_brick_regress_confounds)
opt.regress_confounds.flag_wm            = true;            % Turn on/off the regression of the average white matter signal (true: apply / false : don't apply)
opt.regress_confounds.flag_vent          = true;          % Turn on/off the regression of the average of the ventricles (true: apply / false : don't apply)
opt.regress_confounds.flag_motion_params = true; % Turn on/off the regression of the motion parameters (true: apply / false : don't apply)
opt.regress_confounds.flag_gsc           = false;          % Turn on/off the regression of the PCA-based estimation of the global signal (true: apply / false : don't apply)
opt.regress_confounds.flag_scrubbing     = true;     % Turn on/off the scrubbing of time frames with excessive motion (true: apply / false : don't apply)
opt.regress_confounds.thre_fd            = 0.5;             % The threshold on frame displacement that is used to determine frames with excessive motion in the scrubbing procedure

% Correction of physiological noise (niak_pipeline_corsica)
opt.corsica.sica.nb_comp = 60;    % Number of components estimated during the ICA. 20 is a minimal number, 60 was used in the validation of CORSICA.
opt.corsica.threshold    = 0.15;  % This threshold has been calibrated on a validation database as providing good sensitivity with excellent specificity.
opt.corsica.flag_skip    = 1;     % Skip CORSICA (0: don't skip, 1 : skip). Even if it is skipped, ICA results will be generated for quality-control purposes. The method is not currently considered to be stable enough for production unless it is manually supervised.

% Spatial smoothing (niak_brick_smooth_vol)
opt.smooth_vol.fwhm      = 6;  % Full-width at maximum (FWHM) of the Gaussian blurring kernel, in mm.
opt.smooth_vol.flag_skip = 0;  % Skip spatial smoothing (0: don't skip, 1 : skip)

% how to specify a different parameter for two subjects (here subject1 and subject2)
%  opt.tune(1).subject = 'subject1';
%  opt.tune(1).param.slice_timing.flag_center = true; % Anything that usually goes in opt can go in param. What's specified in opt applies by default, but is overridden by tune.param
%  
%  opt.tune(2).subject = 'subject2';
%  opt.tune(2).param.slice_timing.flag_center = false; % Anything that usually goes in opt can go in param. What's specified in opt applies by default, but is overridden by tune.param

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%% Run the fmri_preprocess pipeline  %%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
opt.flag_test = false;
[pipeline,opt] = niak_pipeline_fmri_preprocess(files_in,opt);

.
27-Jun-2016 14:52:36 t1_preprocess_A1470                       failed     (5 run / 1 fail / 5 done / 1803 left)
27-Jun-2016 14:52:36 slice_timing_A1522_session1_inscape       submitted  (6 run / 1 fail / 5 done / 1802 left)
.Deamon terminated on 27-Jun-2016 14:52:57

server: noisetier
Subject A1470
Subject A1522
Subject A1568
Subject A1587
Subject A1611
Subject A1623
Subject A1642
Subject A1731
Subject A1773
Subject A1790
Subject A1820
Subject A1820B
Subject B1658
Subject B1687
Subject B1692
Subject B1783
Subject C2063
Subject C2065
Subject C2106
Subject D2098
Subject D2228
Subject D2354
Subject D2358
Subject D2410
Subject D2422
Subject D2443
Subject D2443B
Subject D2545
Subject D2568
Subject D2570
Subject D2583
Subject P0003
